<a href="https://colab.research.google.com/github/giteshgoyal/llm_practice/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Vocablury

1. Load the text
2. Split the text using whitespace, punctations as delimiter
3. Remove whitespace from token list (this is optional as we may need whitepace as token)
4. Sort and assign value to token list.

# Load the Text

## Load text using url

In [39]:
import requests

response = requests.get("https://raw.githubusercontent.com/giteshgoyal/llm_practice/refs/heads/main/verdict.txt")
raw_text = response.text

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20480
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Load using cloning the git repo

In [40]:
!git clone https://github.com/giteshgoyal/llm_practice.git

fatal: destination path 'llm_practice' already exists and is not an empty directory.


In [41]:
!ls
with open("llm_practice/verdict.txt", "r", encoding="utf-8") as f:
  raw_text=f.read()

print("Total number of character: ", len(raw_text))
print(raw_text[:99])

llm_practice  sample_data
Total number of character:  20480
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Split the text using whitespace, punctations as delimiter

In [42]:
import re

splited_data=re.split(r'([,.:;?_!"()\']|\s)',raw_text)
print(splited_data[:99])
print(len(splited_data))

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius--though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough--so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', '', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', ',', '', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting', ',', '', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow', ',', '', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on']
9043


## Remove whitespace from token list

In [43]:
cleaned_data= [data.strip() for data in splited_data if data.split()]
print(cleaned_data[:99])
print(len(cleaned_data))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius--though', 'a', 'good', 'fellow', 'enough--so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing--his', 'last', 'Chicago', 'sitter--deploring', 'his', 'unaccountable', 'abdication', '.', '"', 'Of', 'course']
4506


## Sort and assign value to token list.


In [44]:
sorted_data= sorted(set(cleaned_data))
print(len(sorted_data))

1200


In [45]:
vocab= {data:id for id,data in enumerate(sorted_data)}
for w,i in vocab.items():
  print(w,":",i)
  if i >30:
    break

! : 0
" : 1
' : 2
( : 3
) : 4
, : 5
--and : 6
--even : 7
--it : 8
--oh : 9
--she : 10
--that : 11
. : 12
: : 13
; : 14
? : 15
A : 16
Ah : 17
Ah--I : 18
Among : 19
And : 20
Are : 21
Arrt : 22
As : 23
At : 24
Be : 25
Begin : 26
Burlington : 27
But : 28
By : 29
Carlo : 30
Chicago : 31


# Word Based Tokenizer

In [46]:
print("lets start")

lets start
